In [ ]:
import os
import random
from math import exp as exp
import numpy as np
import matplotlib.pyplot as plt

import albumentations as A

import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets, utils
from torchvision.datasets import ImageFolder
from models import resnet_model

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image


In [ ]:
# Setup_seed
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)  # os 자체의 seed 고정
    
setup_seed(42)

In [ ]:
# Hyperparameters & etc.
GPU_NUM = 0
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='4'
print('gpu? ', torch.cuda.is_available())
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

fn_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0,2,3,1)
fn_denorm = lambda x, mean, std : (x * std) + mean 


In [ ]:
# Dataset & Loader config

EvalDataset = ImageFolder(root = '/eval_set',
                            transform=transforms.Compose([
                                transforms.Resize((224,224)),
                                transforms.ToTensor(),
                           ]))
EvalDataLoader = DataLoader(EvalDataset, batch_size = 32, shuffle=True)

# EvalTransForms = A.Compose([    
#     A.pytorch.transforms.ToTensorV2()
# ])
# EvalDataset = Dataset('', EvalTransForms)
# EvalDataLoader = DataLoader(EvalDataset, batch_size = 4, shuffle=True)

In [ ]:
# display sample images
def DisplaySampleImages(Dataset, labels=True, nrow=4, grid_size=4, padding=2, normalize=True):

    grid_size=grid_size
    rnd_inds=np.random.randint(0,len(Dataset),grid_size)

    images=[Dataset[i][0] for i in rnd_inds]

    images=utils.make_grid(images, nrow=nrow, padding=padding, normalize = normalize)


    # call helper function
    plt.figure(figsize=(10,10))
    
    npimg = images.numpy()
    npimg_tr = np.transpose(npimg, (1, 2, 0))
    plt.imshow(npimg_tr)
    if labels is True:
        
        labels=[Dataset[i][1] for i in rnd_inds]
        plt.title('labels: ' + str(labels))
    print("image indices:",rnd_inds)
    print(Dataset[0][0].shape)
    

In [ ]:
# Model config 
num_classes = 2
model = resnet_model(num_classes,pretrained = True)
dicts = torch.load('/.pth')

model.load_state_dict(dicts['net_state_dict'])
epoch = dicts['epoch']
acc = dicts['acc']

model = model.to(device)
print('Model Epoch : ',epoch)
print('Model Accuracy : ',acc)

In [ ]:
##########################  evaluate net and save model  ###############################

model.eval()

LogitArr=[]
LabelArr =[]
PredArr = []
EvalTotal = 0
EvalCorrect = 0
for batch_idx, (img, label) in enumerate(EvalDataLoader):
    with torch.no_grad():
        img, label = img.to(device), label.to(device)
        TotalSize = len(EvalDataLoader.dataset)
        total_batch = len(EvalDataLoader)
        batch_size = len(label)
        
        Logits = model(imgs=img)
        logit = Logits.to('cpu').detach().numpy()
        for i in range(len(logit)):
            LogitArr.append(logit[i][1])
            # LogitArr.append(logit[i][logit[i].argmax()])
            PredArr.append(np.argmax(logit[i],axis=0))
            
        labels = label.to('cpu').detach().numpy()
        for i in range(len(labels)):
            LabelArr.append(labels[i]) 
        
        # calculate accuracy
        
        EvalTotal += batch_size
        EvalCorrect += (torch.max(Logits, 1)[1]==label.data).sum().item()

In [ ]:
# Calculate roauc
 
def CalculateAuc(y_true, y_score):
    fprs , tprs , thresholds = roc_curve(np.array(y_true), np.array(y_score))
    roauc = roc_auc_score(y_true, y_score)
    
    plt.plot(fprs, tprs, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--', label='Random')
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.yticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.xlabel('False Positive Rate (1 - Sensitivity)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()
    print('AUC: {:.4f}'.format(roauc))

In [ ]:

def GetEvalMetrics(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    F1 = f1_score(y_true, y_pred)

    print('Accuracy: {:.4f}'.format(accuracy))
    print('Precision: {:.4f}'.format(precision))
    print('Recall: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('예측값 10개만 표시: ', y_pred[:10])
    

In [ ]:
def ConfusionMatrix(y_true, y_pred, title='Confusion matrix', cmap='Blues',normalize= None):
    confusion = confusion_matrix(y_true, y_pred)
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,cmap=cmap,normalize = normalize )
    plt.title(title)
    plt.show()
    print('Confusion matrix:\n', confusion)
    

In [ ]:
# !pip install grad-cam
def SimpleCam(model, DataLoader, target_layers):
    for i, (img, label) in enumerate(DataLoader):   
        img, label = img.to(device), label.to(device)
        input_tensor = img
        # Create an input tensor image for your model..
        # Note: input_tensor can be a batch tensor with several images!

        # Construct the CAM object once, and then re-use it on many images:
        cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

        # You can also use it within a with statement, to make sure it is freed,
        # In case you need to re-create it inside an outer loop:
        # with GradCAM(model=model, target_layers=target_layers, use_cuda=args.use_cuda) as cam:
        #   ...

        # We have to specify the target we want to generate
        # the Class Activation Maps for.
        # If targets is None, the highest scoring category
        # will be used for every image in the batch.
        # Here we use ClassifierOutputTarget, but you can define your own custom targets
        # That are, for example, combinations of categories, or specific outputs in a non standard model.
        
        targets = None
        # targets = [ClassifierOutputTarget(1)]
        # targets = [ClassifierOutputTarget(0)]
        
        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

        # In this example grayscale_cam has only one image in the batch:
        grayscale_cam = grayscale_cam[0, :]
        np_arr = fn_tonumpy(img)
        np_arr = np_arr[0,:]
        visualization = show_cam_on_image(np_arr, grayscale_cam, use_rgb=True)
        fig = plt.figure()
        rows = 1
        cols = 2
        ax1 = fig.add_subplot(rows, cols, 1)
        ax1.axis("off")
        plt.imshow(np_arr)
        ax2 = fig.add_subplot(rows, cols, 2)
        plt.imshow(visualization)
        ax2.axis("off")
        plt.show() 

In [ ]:
GetEvalMetrics(LabelArr,PredArr)

CalculateAuc(LabelArr, LogitArr)

DisplaySampleImages(EvalDataset)

# pip install -U scikit-learn
ConfusionMatrix(LabelArr,PredArr)

In [ ]:
SimpleCam(model, EvalDataLoader,[model.model.layer4])